In [13]:
# tf-idf for creative_id and advertiser_id
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

train_merged=pd.read_csv('train_merged.csv')
v = TfidfVectorizer(lowercase=False,tokenizer=lambda x:x)

creative_id=train_merged.groupby("user_id")["creative_id"].apply(list)
creative_id = v.fit_transform(creative_id)
advertiser_id=train_merged.groupby("user_id")["advertiser_id"].apply(list)
advertiser_id = v.fit_transform(advertiser_id)
cr_ad=sparse.hstack((creative_id,advertiser_id),dtype=float)
display(cr_ad)
sparse.save_npz('cr_ad.npz', cr_ad)

<900000x2533225 sparse matrix of type '<class 'numpy.float64'>'
	with 48745802 stored elements in COOrdinate format>

In [1]:
# predict gender
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse

user_info=pd.read_csv('user_info.csv')
creative_matrix=sparse.load_npz('cr_ad.npz')
X_train=creative_matrix
y_train=user_info["gender"]
display(X_train,y_train)
train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('gender.bin')

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


<900000x2451045 sparse matrix of type '<class 'numpy.float64'>'
	with 10312696 stored elements in COOrdinate format>

0         0
1         0
2         1
3         0
4         0
         ..
899995    0
899996    1
899997    1
899998    0
899999    1
Name: gender, Length: 900000, dtype: int64

In [1]:
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd

train_data = lgb.Dataset('gender.bin')
lgb_params = {'num_leaves': 2**5-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
print("start cv")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, shuffle=True,
    verbose_eval=1, show_stdv=True)

start cv
[1]	cv_agg's auc: 0.767025 + 0.000449946
[2]	cv_agg's auc: 0.826188 + 0.00121602
[3]	cv_agg's auc: 0.845422 + 0.000863808
[4]	cv_agg's auc: 0.851712 + 0.000605036
[5]	cv_agg's auc: 0.858422 + 0.00167923
[6]	cv_agg's auc: 0.866107 + 0.00223791
[7]	cv_agg's auc: 0.868168 + 0.00215166
[8]	cv_agg's auc: 0.869815 + 0.00173691
[9]	cv_agg's auc: 0.875981 + 0.00172246
[10]	cv_agg's auc: 0.879514 + 0.00162535
[11]	cv_agg's auc: 0.883363 + 0.00181844
[12]	cv_agg's auc: 0.885146 + 0.00141157
[13]	cv_agg's auc: 0.888497 + 0.0013869
[14]	cv_agg's auc: 0.890701 + 0.00106163
[15]	cv_agg's auc: 0.893231 + 0.000907936
[16]	cv_agg's auc: 0.894663 + 0.000914995
[17]	cv_agg's auc: 0.896867 + 0.00099255
[18]	cv_agg's auc: 0.898301 + 0.000900997
[19]	cv_agg's auc: 0.899725 + 0.000639843
[20]	cv_agg's auc: 0.901443 + 0.000951029
[21]	cv_agg's auc: 0.903209 + 0.000805491
[22]	cv_agg's auc: 0.904704 + 0.00076453
[23]	cv_agg's auc: 0.906559 + 0.000772696
[24]	cv_agg's auc: 0.907973 + 0.000716829
[25]	c

KeyboardInterrupt: 

# using test data to train

In [1]:
# predict gender
# tf-idf for creative_id
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

predict_merged=pd.read_csv('predict_merged.csv')
v = TfidfVectorizer(lowercase=False,tokenizer=lambda x:x)

creative_id=predict_merged.groupby("user_id")["creative_id"].apply(list)
creative_id = v.fit_transform(creative_id)
advertiser_id=predict_merged.groupby("user_id")["advertiser_id"].apply(list)
advertiser_id = v.fit_transform(advertiser_id)
pred_cr_ad=sparse.hstack((creative_id,advertiser_id),dtype=float)
display(pred_cr_ad)
sparse.save_npz('pred_cr_ad.npz', pred_cr_ad)

<1000000x2671020 sparse matrix of type '<class 'numpy.float64'>'
	with 54252557 stored elements in COOrdinate format>

In [ ]:
# predict gender
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse
from sklearn.model_selection import KFold
import numpy as np

user_info=pd.read_csv("user_info.csv")
X_train=sparse.load_npz('cr_ad.npz')
y_train=user_info["gender"]
X_test=sparse.load_npz('pred_cr_ad.npz')
display(X_train.shape,X_test.shape)
diff=X_test.shape[1]-X_train.shape[1]
zero_matrix=sparse.csr_matrix((X_train.shape[0],diff))
X_train=sparse.hstack((X_train,zero_matrix)).tocsr()
display(X_train,X_test)
predictions = np.zeros(X_test.shape[0])

folds = KFold(n_splits=5, shuffle=False)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    display(trn_idx,val_idx)
    trn_data_X_train=X_train.tocsr()[trn_idx,:]
    val_data_X_train=X_train.tocsr()[val_idx,:]
    display(trn_data_X_train,val_data_X_train)
    trn_data = lgb.Dataset(trn_data_X_train, y_train.iloc[trn_idx])
    val_data = lgb.Dataset(val_data_X_train, y_train.iloc[val_idx])
    
    lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
    model = lgb.train(lgb_params, 
                    trn_data,
                    num_boost_round=200, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval = 10, 
                    early_stopping_rounds = 100)
#     display(model.predict(X_test, num_iteration=model.best_iteration) / folds.n_splits)
    predictions += model.predict(X_test, num_iteration=model.best_iteration) / folds.n_splits
    display(predictions)

display(predictions)

(900000, 2533225)

(1000000, 2671020)

<900000x2671020 sparse matrix of type '<class 'numpy.float64'>'
	with 48745802 stored elements in Compressed Sparse Row format>

<1000000x2671020 sparse matrix of type '<class 'numpy.float64'>'
	with 54252557 stored elements in COOrdinate format>

fold n°1


...

array([     0,      1,      2, ..., 899997, 899998, 899999])

array([540000, 540001, 540002, ..., 719997, 719998, 719999])

<720000x2671020 sparse matrix of type '<class 'numpy.float64'>'
	with 38998989 stored elements in Compressed Sparse Row format>

<180000x2671020 sparse matrix of type '<class 'numpy.float64'>'
	with 9746813 stored elements in Compressed Sparse Row format>

Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.911609	valid_1's auc: 0.910397
[20]	training's auc: 0.924172	valid_1's auc: 0.92315
[30]	training's auc: 0.934208	valid_1's auc: 0.932896
[40]	training's auc: 0.941389	valid_1's auc: 0.939887


In [7]:
df=pd.DataFrame(predictions,columns=["gender"])

In [9]:
df[df.gender>=0.5]

,gender
50,0.649246
244,0.700756
1948,0.675952
2091,0.502977
2185,0.503331
...,...
998902,0.507344
998940,0.633742
999070,0.501990
999091,0.856675


In [10]:
submission=(df>0.5)*1

In [21]:
submission

,gender,user_id,age
0,1,3000001,1
1,1,3000002,1
2,1,3000003,1
3,1,3000004,1
4,1,3000005,1
...,...,...,...
999995,1,3999996,1
999996,1,3999997,1
999997,1,3999998,1
999998,1,3999999,1


In [22]:
submission["gender"]=[0 for i in range(3000001,4000001)]

In [25]:
submission["age"]=[0 for _ in range(1000000)]

In [27]:
submission.describe()

,gender,user_id,age
count,1000000.0,1.000000e+06,1000000.0
mean,0.0,3.500000e+06,0.0
std,0.0,2.886753e+05,0.0
min,0.0,3.000001e+06,0.0
25%,0.0,3.250001e+06,0.0
50%,0.0,3.500000e+06,0.0
75%,0.0,3.750000e+06,0.0
max,0.0,4.000000e+06,0.0


In [28]:
submission.to_csv("submission.csv",index=False)

In [1]:
%reset -f
# !pip install lightgbm
import lightgbm as lgb
train_data = lgb.Dataset('gender.bin')
lgb_params = {'num_leaves': 2**5-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
print("start training")
model = lgb.train(lgb_params, train_data,num_boost_round=120)
model.save_model("gender_model.txt")
print("training completed!")

start training
training completed!


In [25]:
import lightgbm as lgb
from scipy import sparse

X_pred=creative_id_matrix
zero_matrix=sparse.csc_matrix((1000000,2533238-2497542))
X_pred=sparse.hstack([X_pred,zero_matrix],dtype=float).tocsr()
print("start predicting")
model=lgb.Booster(model_file='gender_model.txt')
y_pred=model.predict(X_pred)
y_pred=(y_pred>0.5)*1
display(y_pred)
submission=pd.DataFrame([i for i in range(3000001,4000001)],columns=["user_id"])
submission["predicted_gender"]=y_pred
display(submission)

start predicting


array([0, 0, 0, ..., 0, 0, 0])

,user_id,predicted_gender
0,3000001,0
1,3000002,0
2,3000003,0
3,3000004,0
4,3000005,0
...,...,...
999995,3999996,0
999996,3999997,0
999997,3999998,0
999998,3999999,0


In [30]:
submission["predicted_age"]=[1 for _ in range(int(1e6))]

In [37]:
submission["predicted_gender"]+=1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype
---  ------            --------------    -----
 0   user_id           1000000 non-null  int64
 1   predicted_gender  1000000 non-null  int64
 2   predicted_age     1000000 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB


In [38]:
submission.to_csv("submission.csv",index=False)